<div align="center">
    <img src= "/assets/content/datax_logos/DataX_blue_wide_logo.png" align="center" width="100%">
</div>

### **HOMEWORK** 03 - **FLASK** DATABASE **DEPLOYMENT**

<br>

<div align="center" style="font-size:12px; font-family:FreeMono; font-weight: 100; font-stretch:ultra-condensed; line-height: 1.0; color:#2A2C2B">
    <img src="/assets/content/images/flask_logo.png" align="center" width="20%" padding="10"><br>
    <br>
</div>

<br>

<br>

In **m320_hw_1--flask_minimal_application** we introduced [**routing (decorators)**](https://flask.palletsprojects.com/en/1.1.x/api/#flask.Flask.route), the [**extends**](https://flask.palletsprojects.com/en/1.1.x/patterns/templateinheritance/) statement, and how to build a **Flask package**. The work on **m320_hw_2--flask_wtf** covered [**web forms** and **form validation**](https://flask.palletsprojects.com/en/1.1.x/patterns/wtforms/). This notebook covers Flask [**database server deployment**](https://flask.palletsprojects.com/en/1.1.x/deploying/#deployment), instead of the more barebones [**SQLite**](https://flask.palletsprojects.com/en/1.1.x/patterns/sqlite3/) disk-based database system used in the introductory coursework. 

In order to successfully complete this assignment, make sure you've watched the myFlaskApp.py walkthrough video, as it discusses in a general way what you are being asked to do.
<br>

**Author list:** [Elias Casto Hernandez](https://www.linkedin.com/in/ehcastroh/)

**References and Additional Resources**
<br>


> * [Flask by Example by Real Python](https://realpython.com/flask-by-example-part-1-project-setup/#project-setup)

> * [Flask Mega Tutorial by Miguel Grinberg](https://courses.miguelgrinberg.com/courses/flask-mega-tutorial/lectures/5203689)

> * [Flask Database Server Options](https://flask.palletsprojects.com/en/1.1.x/tutorial/database/?highlight=database)

<hr style="border: 2px solid#003262;" />

#### HW 3: FLASK + POSTGRESQL

<div align="center" style="font-size:12px; font-family:FreeMono; font-weight: 100; font-stretch:ultra-condensed; line-height: 1.0; color:#2A2C2B">
    <img src="/assets/content/images/flask-03.png" align="center" width="30%" padding="10"><br>
    <br>
</div>

<br>

**Note:** there are various database(s) solutions to the particularities of your Flask implementation. Since the [Data-X](https://datax.berkeley.edu/) curriculum is focused on applied machine learning (and applied data science), we will be deploying a [**PostgreSQL**](https://www.postgresql.org/) database as it is commonly found in contemporary machine learning pipelines. Moreover, the next homework -- **m320_hw_4--flask_on_heroky** -- is deployed on Heroku, which uses Postgres. See [here](https://flask.palletsprojects.com/en/1.1.x/tutorial/database/?highlight=database) to learn how to connect Flask to your database of choice.

This assignment assumes that hw_1 and hw_2 are fully working as requested, in *local host*.

```bash
/myFlaskPackage
/venv
/flaskApp
/__init__.py
/routes.py
/config.py
/manage.py         # created in hw_3
/templates
    /base.html
    /index.html
    /login.html
/static
    /CSS
        /main.css
```

___

### **TO-DO**

0) Install PostgreSQL

https://www.postgresql.org/download/

```bash
# in local machine
$ sudo apt-get install postgresql
```

or

```bash
# in a virtual environment install sqlalchemy
(venv) $ pip install flask-sqlalchemy
```

___

**Note:** If you need help initializing your virtual environment, see **m320_Setting-up-Flask** for instructions on how to create the environment.
___

<br>

00) Install Flask-Migrate 

https://github.com/miguelgrinberg/flask-migrate


```bash
# make sure you are in a virtual environment 
(venv) $ pip install flask-migrate
```

<br>

1) Create a PostgreSQL database

```sql 
$ psql
# create database mypostgresDB;
CREATE DATABASE
# \q
```


<br>

2) Update configuration, ```config.py```, file -- it should now look like this:

```python
import os
basedir = os.path.abspath(os.path.dirname(__file__))

class Config(object):
    DEBUG = False
    TESTING = False
    CSRF_ENABLED = True
    SECRET_KEY = os.environ.get('SECRET_KEY') or 'never-gonna-guess'
    SQLALCHEMY_DATABASE_URI = os.environ['DATABASE_URL']

# configure production
class ProductionConfig(Config):
    DEBUG = FALSE

# configure staging
class StagingConfig(Config):
    Development = True
    Debug = True
    
class DevelopmentConfig(Config):
    DEVELOPMENT = True
    DEBUG = True

class TestingConfig(Config):
    TESTING = True
```

<br>

3) Add ```DATABASE_URL``` variable to ```mypostgresDB``` database (done in local environment)

```shell
$ export DATABASE_URL="postgresql:///mypostgresDB"
```

<br>

4) Update the ```__init__.py``` file to contain the following:

```python
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from config import Config
from flask_migrate import Migrate
import os

app = Flask(__name__)
app.config.from_object(Config)
# initialize database by passing app object defined above
db = SQLAlchemy(app)
migrate = Migrate(app, db)

app.config.from_object(Config)
app.config.from_object(os.environ['APP_SETTINGS'])
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

from models import Result
from app import routes, model


```
    

<br>

5) create new module ```models.py``` for creating data entries, and enter the following:

```python
from datetime import datetime
from app import db

# create class and define initial database schema
class userEmail(db.Model):
	# create visit id
	id = db.Column(db.Integer, primary_key=True)
	# request user name
	username = db.Column(db.String(64), index=True, unique=True)
	# set max-characters to 200 and require that field be completed
	email = db.Column(db.String(200), nullable = False, unique=True)
	# inaccessible column to keep track of submissions
	completed = db.Column(db.Integer, default = 0)
	# track date of email submission
	date_submitted = db.Column(db.DateTime, default=datetime.utcnow)
    posts = db.relationship('Post', backref='author', lazy='dynamic')

	# function to return notice of data entry
	def __repr__(self):
		# string formatting converts the self.id value to a string using repr()
		return '<userEmail {}>'.format(self.email)

# create class for updating database entries (blog)    
class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    body = db.Column(db.String(140))
    timestamp = db.Column(db.DateTime, index=True, default=datetime.utcnow)
    # references id value from user's table
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))

    # return string of posted body content
    def __repr__(self):
        return '<Post {}>'.format(self.body)
```

<br>

6) Create a new file name ```manage.py``` to handle database migrations. It should contain the following:

```python
import os
from flask_script import Manager
from flask_migrate import Migrate, MigrateCommand

from app import app, db


app.config.from_object(os.environ['APP_SETTINGS'])

migrate = Migrate(app, db)
manager = Manager(app)

manager.add_command('db', MigrateCommand)


if __name__ == '__main__':
    manager.run()

```

<br>

7) Run migrations using Alembic and Migrate

>Initialize database
```bash
$ python manage.py db init
```

**Note:** running this database initialization command will create a new folder in the working directory, as specified in item 3). If unsure of what is happening up-to this point, what the **myFlaskApp.py** walkthrough video as it will explain a simplified version of this process.

>Migrate database
```bash
$ python manage.py db migrate
```

>Upgrade database
```bash
$ python manage.py db upgrade
```

<br>

<hr style="border: 2px solid#003262;" />

### **DELIVERABLES**

Please submit the following: 

>(1) Use the **m320_Setting-up-Flask** materials to create a virtual environment for work to be done.

>(2) Create a python prompt like in Concept Check Two in the **m320_Setting-up-Flask** notebook. Then enter the following:

```python
>>>from app import db
>>>from app.models import userEmail, Post
# create a new user
>>> u = userEmail(username='Elias', email='elias@example.com')
```

In a few sentences, explain what happens when you execute the following commands ```db.session.add(u)```, ```db.session.commit()```, and ```db.session.rollback()```

>(3) Add as many new users as you want using deliverable 2 as an example. Then query the user names by iterating over the following object. Provide a screen capture of this step.
```python
>>> all_users = userEmail.query.all()
>>> all_users # what is the output here?
```


>(4) Enter the following command 

```sql
    $ psql
    # \c mypostgresDB
```

you should receive a response along the lines of 

```You are now connected to database "mypostgresDB"```...



Once you receive the message above, enter the following two commands, do screen capture(s) for each, and submit them with your complete directory and files:
    
```sql
# \dt
```

and 

```sql
# \d results
```

<hr style="border: 2px solid#003262;" />